In [3]:
import os
from dotenv import load_dotenv
load_dotenv()

groq_api_key = os.getenv("GROQ_API_KEY")
groq_api_key

'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'

In [4]:
from langchain_groq import ChatGroq
model = ChatGroq(model="Gemma2-9b-It", groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001C7C9E3C200>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001C7C9E3CD40>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [5]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hi my name is Shriju and I am an AI Engineer")])

AIMessage(content="Hi Shriju, it's nice to meet you!  \n\nAs an AI, I don't have personal experiences like being an engineer, but I'm always eager to learn more about what you do. What kind of AI projects are you working on?  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 59, 'prompt_tokens': 21, 'total_tokens': 80, 'completion_time': 0.107272727, 'prompt_time': 0.00014436, 'queue_time': 0.02327023, 'total_time': 0.107417087}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-691a1cda-9c8d-464e-94e8-d5cf5bd99174-0', usage_metadata={'input_tokens': 21, 'output_tokens': 59, 'total_tokens': 80})

In [6]:
from langchain_core.messages import AIMessage
model.invoke(
    [
        HumanMessage(content=" Hi, my name is Shriju, I am an AI Engineer"),
        AIMessage(content="Hi Shriju, it is nice to meet you! \n\nAs an AI, I don't have personal experiences like being an engineer, but I'm always eager to learn more about what you do. What are you working on these days?"),
        HumanMessage(content = "Hey what is my name and what do I do?")
    ]
)

AIMessage(content="You are Shriju, and you are an AI Engineer! 😊  \n\nIs there anything else you'd like to know or talk about? I'm here to help.\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 39, 'prompt_tokens': 94, 'total_tokens': 133, 'completion_time': 0.070909091, 'prompt_time': 0.002541031, 'queue_time': 0.011147717000000001, 'total_time': 0.073450122}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-0290f459-909a-45de-977e-7df9980078eb-0', usage_metadata={'input_tokens': 94, 'output_tokens': 39, 'total_tokens': 133})

In [7]:
### message History
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store={}

def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]

with_message_history = RunnableWithMessageHistory(model, get_session_history)

In [8]:
config={"configurable":{"session_id": "chat1"}}

In [9]:
response= with_message_history.invoke(
    [HumanMessage(content="Hi, my name is Shriju and I am an AI Engineer")],
    config=config
)

In [10]:
response.content

"Hello Shriju,\n\nIt's nice to meet you! As an AI myself, I'm always interested in talking to other people who work in the field. \n\nWhat kind of AI engineering do you specialize in?  Do you have any interesting projects you're working on that you'd like to tell me about?\n"

In [11]:
with_message_history.invoke(
    [HumanMessage(content="What is my name?")],
    config=config
)

AIMessage(content='Your name is Shriju.  I remember that you told me at the beginning of our conversation. 😊  \n\nIs there anything else I can help you with?\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 36, 'prompt_tokens': 106, 'total_tokens': 142, 'completion_time': 0.065454545, 'prompt_time': 0.003244487, 'queue_time': 0.013624068999999999, 'total_time': 0.068699032}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-03c7a62e-2538-4690-b6a7-009242a654e0-0', usage_metadata={'input_tokens': 106, 'output_tokens': 36, 'total_tokens': 142})

In [12]:
## change the config--> session ID
config1={"configurable":{"session_id": "chat2"}}
response = with_message_history.invoke(
    [HumanMessage(content="What is my name?")],
    config=config1
)
response.content

"As an AI, I have no memory of past conversations and do not know your name. If you'd like to tell me, I'd be happy to use it!\n"

In [13]:
response = with_message_history.invoke(
    [HumanMessage(content="My name is Shreyasi?")],
    config=config1
)
response.content

"Hello Shreyasi! It's nice to meet you. 😊\n\nIs there anything I can help you with today?\n"

In [14]:
response = with_message_history.invoke(
    [HumanMessage(content="What is my name?")],
    config=config1
)
response.content

'Your name is Shreyasi.  \n\nI remember you told me! 😊  Is there anything else I can help you with?\n'

In [16]:
## Prompt templates
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant. Answer all the question to the best of your ability in {language}."),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain=prompt|model

In [17]:
response = chain.invoke({"messages":[HumanMessage(content="My name is Shriju")], "language":"Hindi"})
response.content

'नमस्ते श्रीजू! 😊  मैं आपकी सहायता करने के लिए यहाँ हूँ। आप मुझसे कोई भी प्रश्न पूछ सकते हैं और मैं अपनी पूरी क्षमता से उत्तर देने की कोशिश करूँगा। \n\n'

In [18]:
chain.invoke({"messages": [HumanMessage(content="My name is Shriju")]})


KeyError: "Input to ChatPromptTemplate is missing variables {'language'}.  Expected: ['language', 'messages'] Received: ['messages']\nNote: if you intended {language} to be part of the string and not a variable, please escape it with double curly braces like: '{{language}}'.\nFor troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/INVALID_PROMPT_INPUT"

In [19]:
with_message_history=RunnableWithMessageHistory(chain, get_session_history)

In [20]:
config = {"configurable":{"session_id": "chat3"}}
response=with_message_history.invoke(
    [HumanMessage(content="My name is Shriju")],
    config=config
)

response.content

TypeError: Expected mapping type as input to ChatPromptTemplate. Received <class 'list'>.
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/INVALID_PROMPT_INPUT

In [21]:
with_message_history=RunnableWithMessageHistory(
    chain, 
    get_session_history,
    input_messages_key="messages")

In [22]:
config={"configurable":{"session_id": "chat4"}}
response=with_message_history.invoke(
    {'messages':[HumanMessage(content="Hi, My name is Shriju")], "language": "Hindi"},
    config=config,
    
)

response.content

'नमस्ते श्रीजू! 👋 \n\nमुझे बहुत खुशी है कि आप मुझसे बात कर रहे हैं। मैं आपकी सहायता करने के लिए तैयार हूँ।  \n\nआप मुझसे क्या पूछना चाहते हैं? 😊\n\n'

In [28]:
# managing the conversation history, trim specifies how many tokens we want to keep
from langchain_core.messages import SystemMessage, trim_messages
trimmer = trim_messages(
max_tokens = 80,
strategy = "last",
token_counter = model,
include_system = True,
allow_partial = False,
start_on = "human"
)
messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="Hi, My name is Shriju"),
    AIMessage(content="hi"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="What is  2 + 2?"),
    AIMessage(content="4"),
    HumanMessage(content="Thanks"),
    AIMessage(content="No problem"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]
trimmer.invoke(messages)

[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Hi, My name is Shriju', additional_kwargs={}, response_metadata={}),
 AIMessage(content='hi', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='What is  2 + 2?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='No problem', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [29]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough
chain=(
    RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer)
    | prompt
    | model
)
response = chain.invoke(
    {
    "messages":messages + [HumanMessage(content="what icecream do i like?")],
    "language":"English"
    }
)
response.content

'You said you like vanilla ice cream! 😊  🍦  Anything else I can help you with?\n'

In [30]:
from langchain_core.documents import Document
documents=[
    Document(
        page_content="Dogs are great companions, known for their loyalty and friendliness.",
        metadata={"source":"mammel-pets-doc"},
    ),
    Document(
        page_content="Cats are independent pets that often enjoy their own space.",
        metadata={"source":"mammel-pets-doc"},
    ),
    Document(
        page_content="Goldfish are popular pets for beginners, returning relatively simple care.",
        metadata={"source":"bird-pets-doc"},
    ),
    Document(
        page_content="Parrots are intelligent birds capable of human mimicking.",
        metadata={"source":"bird-pets-doc"},
    ),
    Document(
        page_content="Rabbits are social animals that need plenty of space to hop around.",
        metadata={"source":"mammel-pets-doc"},
    ),
]




In [31]:
documents

[Document(metadata={'source': 'mammel-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.'),
 Document(metadata={'source': 'mammel-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
 Document(metadata={'source': 'bird-pets-doc'}, page_content='Goldfish are popular pets for beginners, returning relatively simple care.'),
 Document(metadata={'source': 'bird-pets-doc'}, page_content='Parrots are intelligent birds capable of human mimicking.'),
 Document(metadata={'source': 'mammel-pets-doc'}, page_content='Rabbits are social animals that need plenty of space to hop around.')]

In [32]:
import os
from dotenv import load_dotenv
load_dotenv()
from langchain_groq import ChatGroq
groq_api_key=os.getenv("GROQ_API_KEY")

os.environ["HF_TOKEN"]=os.getenv("HF_TOKEN")

llm= ChatGroq(groq_api_key=groq_api_key, model="Llama3-8b-8192")
llm


ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001C7EAD2F3B0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001C7E813D160>, model_name='Llama3-8b-8192', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [33]:
#embedding
from langchain_huggingface import HuggingFaceEmbeddings
embeddings=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

In [34]:
# vectorstores
#from langchain_chroma import Chroma
from langchain.vectorstores import FAISS

## vectorstore=Chroma.from_documents(documents, embedding=embeddings)
vectorstore = FAISS.from_documents(documents, embedding=embeddings)
vectorstore


In [35]:
vectorstore.similarity_search("cat")

[Document(metadata={'source': 'mammel-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
 Document(metadata={'source': 'mammel-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.'),
 Document(metadata={'source': 'mammel-pets-doc'}, page_content='Rabbits are social animals that need plenty of space to hop around.'),
 Document(metadata={'source': 'bird-pets-doc'}, page_content='Parrots are intelligent birds capable of human mimicking.')]

In [37]:
# Async query
await vectorstore.asimilarity_search("cat")

[Document(metadata={'source': 'mammel-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
 Document(metadata={'source': 'mammel-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.'),
 Document(metadata={'source': 'mammel-pets-doc'}, page_content='Rabbits are social animals that need plenty of space to hop around.'),
 Document(metadata={'source': 'bird-pets-doc'}, page_content='Parrots are intelligent birds capable of human mimicking.')]

In [38]:
vectorstore.similarity_search_with_score("cat")

[(Document(metadata={'source': 'mammel-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
  0.9351057),
 (Document(metadata={'source': 'mammel-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.'),
  1.5740898),
 (Document(metadata={'source': 'mammel-pets-doc'}, page_content='Rabbits are social animals that need plenty of space to hop around.'),
  1.5956905),
 (Document(metadata={'source': 'bird-pets-doc'}, page_content='Parrots are intelligent birds capable of human mimicking.'),
  1.6411983)]

In [39]:
from typing import List

from langchain_core.documents import Document
from langchain_core.runnables import RunnableLambda

retriever=RunnableLambda(vectorstore.similarity_search).bind(k=1)
retriever.batch(["cat", "dog"])

[[Document(metadata={'source': 'mammel-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.')],
 [Document(metadata={'source': 'mammel-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.')]]

In [40]:
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 1},
    
)
retriever.batch(["cat", "dog"])

[[Document(metadata={'source': 'mammel-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.')],
 [Document(metadata={'source': 'mammel-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.')]]

In [43]:
# integrate retriever with chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

message = """
Answer this quesgtion using the provided context only.

{question}

Context: 
{context}
"""
prompt = ChatPromptTemplate.from_messages([("human", message)])

rag_chain={"context":retriever, "question":RunnablePassthrough()}|prompt|llm

response = rag_chain.invoke("tell me about dogs")
print(response.content)

Based on the provided context, here's what I can tell you about dogs:

Dogs are great companions, known for their loyalty and friendliness.


In [ ]:
#145/28 .\env\Scripts\Activate